# Web scraping real estate data from Bogota, Colombia
[Author: Elias Buitrago Bolivar](https://github.com/ebuitrago?tab=repositories)

This jupyter notebook depicts a python based web scraping  algorithm to obtain real estate data from the portal fincaraiz.com.co. The code presented here is functional and was tested by scraping real estate data of used apartments sales from Bogota, Colombia. It is recommended to run the notebook locally in Ananconda, as there is a bug when running it on platforms such as google colab.

_Updated: February 25th, 2023_


## Experimental design

In [2]:
!pip install lxml
!pip install scrapy
!pip3 install requests-html
!pip3 install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

### Web Scraping real estate data
This section explains the web scraping process applied to the fincaraiz.com.co web page.

In [3]:
!pip install undetected_chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for undetected_chromedriver: filename=undetected_chromedriver-3.5.0-py3-none-any.whl size=45155 sha256=3f8706257a4e80c4303c3d529dea5eac6e1b4c8b7598dde029e6fed1c15adfb7
  Stored in directory: /root/.cache/pip/wheels/54/4a/ba/5912509dd11b521b9cb2b633cf81936ad47cc92d9668da017f
Successfully built undetected_chromedriver


#### Import required libreries

In [1]:
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import time
import random
import pandas as pd

In [2]:
#Function to get 'href' from each article item
def gethref(soup):

    links = []
    for article in soup.find_all('article'):
        url = article.find('a', href=True)
        if url:
            link = url['href']
            links.append(link)
    print("Href obtained: ", len(links))

    return links

In [3]:
#Function to remove "/proyectos-de-vivienda" articles
def remove_proyectos(links):

    ind = [x.find("/proyecto-de-vivienda") for x in links]
    ind2del = []
    i=0
    for x in ind:
        if x == 0:
            ind2del.append(i)
        i = i +1

    id_inmuebles_todelete = []
    [id_inmuebles_todelete.append(links[x]) for x in ind2del]
    print("Articles '/proyecto-de-vivienda' identified: ", len(id_inmuebles_todelete))

    id_inmuebles = 0
    id_inmuebles = links
    for i in range(len(id_inmuebles_todelete)):
        id_inmuebles.remove(id_inmuebles_todelete[i])

    return id_inmuebles

In [4]:
#Function to call housing_features routine on each href
def scrapper(id_inmueble):

    #Initialize and execute Selenium
    browser = uc.Chrome()
    url_inm = 'https://www.fincaraiz.com.co'+ id_inmueble + '/'
    print(url_inm)
    browser.get(url_inm)
    browser.implicitly_wait(10)
    html=browser.page_source

    #Obtaining the html from the web page after applying Selenium
    soup=bs(html,'lxml')

    #Create a list to store info obtained from one particular property
    features = []

    #Applying function to obtain variables defined from one particular property
    features = housing_features(soup)

    #Close the web browser tab
    browser.close()

    #Close the web browser
    browser.quit()

    return(features)

In [5]:
# Version 1.0
def housing_features(soup):

    #Obtaining whole info from the html section that stores main housing variables
    s = soup.find('section',{'class':'jss9'}).find_all('p')
#     print(s)

    #Extract first 10 features from soup
    plist = varsfromscrap(s)

    #Extracting and adding name to features list
    aux1 = soup.find('header',{'class': 'jss116'}).find_all('p')[0].text.split(' ')
    plist.append(aux1[0])

    #Extracting and adding location to features list
    p = soup.find('header',{'class': 'jss116'}).find_all('p')[1].text.split('-')
    plist.append(p[0].strip())

    #Extracting and adding price to features list
    #Verification to find the price because the card may include a header <p>with news about the price.
    #For instance: "El precio bajó recientemente".
    #That kind of label shifts the position of the <p> corresponding to the price.
    try:
        p=soup.find('div',{'class': 'jss11'}).find_all('p')[1].text
        aux = p.replace('$\xa0','').replace('.','')
        plist.append(int(aux))
    except:
        p=soup.find('div',{'class': 'jss11'}).find_all('p')[2].text
        aux = p.replace('$\xa0','').replace('.','')
        plist.append(int(aux))

    return plist

In [6]:
#Function to extract first 10 features from soup
def varsfromscrap(soup):

    features = [0]*28

    #Transform from bs4.element.ResultSet to list
    plist = []
#     for i in range(len(soup)-1):
    for j in range(len(soup)-3):
        plist.append(soup[j].text)

    #***Habitaciones***
    try:
        i = plist.index("Habitaciones")
        features[0] = plist[i+1]
    except:
        features[0] = 'No definida'

    #***Baños***
    try:
        i = plist.index("Baños")
        features[1] = plist[i+1]
    except:
        features[1] = 'No definida'

    #***Parqueaderos***
    try:
        i = plist.index("Parqueaderos")
        features[2] = plist[i+1]
    except:
        features[2] = '0'

    #***Área construída***
    try:
        i = plist.index("Área construída")
        features[3] = plist[i+1]
    except:
        features[3] = 'No definida'

    #***Área privada***
    try:
        i = plist.index("Área privada")
        features[4] = plist[i+1]
    except:
        features[4] = 'No definida'

    #***Estrato***
    try:
        i = plist.index("Estrato")
        features[5] = plist[i+1]
    except:
        features[5] = 'No definida'

    #***Estado***
    try:
        i = plist.index("Estado")
        features[6] = plist[i+1]
    except:
        features[6] = 'No definida'

    #***Antigüedad***
    try:
        i = plist.index("Antigüedad")
        features[7] = plist[i+1]
    except:
        features[7] = 'No definida'

    #***Administración***
    try:
        i = plist.index("Administración")
        features[8] = plist[i+1]
    except:
        features[8] = 'No definida'

    #***Precio m²***
    try:
        i = plist.index("Precio m²")
        features[9] = plist[i+1]
    except:
        features[9] = 'No definida'

    #***Ascensor***
    try:
        i = plist.index("Ascensor")
        features[10] = 1
    except:
        features[10] = 0

    #***Circuito cerrado de TV***
    try:
        i = plist.index("Circuito cerrado de TV")
        features[11] = 1
    except:
        features[11] = 0

    #***Parqueadero Visitantes***
    try:
        i = plist.index("Parqueadero Visitantes")
        features[12] = 1
    except:
        features[12] = 0

    #***Portería / Recepción***
    try:
        i = plist.index("Portería / Recepción")
        features[13] = 1
    except:
        features[13] = 0

    #***Zonas Verdes***
    try:
        i = plist.index("Zonas Verdes")
        features[14] = 1
    except:
        features[14] = 0

     #***Salón Comunal***
    try:
        i = plist.index("Salón Comunal")
        features[15] = 1
    except:
        features[15] = 0

     #***Balcón***
    try:
        i = plist.index("Balcón")
        features[16] = 1
    except:
        features[16] = 0

     #***Barra estilo americano***
    try:
        i = plist.index("Barra estilo americano")
        features[17] = 1
    except:
        features[17] = 0

     #***Calentador***
    try:
        i = plist.index("Calentador")
        features[18] = 1
    except:
        features[18] = 0

     #***Chimenea***
    try:
        i = plist.index("Chimenea")
        features[19] = 1
    except:
        features[19] = 0

    #***Citófono***
    try:
        i = plist.index("Citófono")
        features[20] = 1
    except:
        features[20] = 0

    #***Cocina Integral***
    try:
        i = plist.index("Cocina Integral")
        features[21] = 1
    except:
        features[21] = 0

    #***Terraza***
    try:
        i = plist.index("Terraza")
        features[22] = 1
    except:
        features[22] = 0

    #***Vigilancia***
    try:
        i = plist.index("Vigilancia")
        features[23] = 1
    except:
        features[23] = 0

    #***Parques cercanos***
    try:
        i = plist.index("Parques cercanos")
        features[24] = 1
    except:
        features[24] = 0

     #***Estudio***
    try:
        i = plist.index("Estudio")
        features[25] = 1
    except:
        features[25] = 0

     #***Patio***
    try:
        i = plist.index("Patio")
        features[26] = 1
    except:
        features[26] = 0

     #***Depósito / Bodega***
    try:
        i = plist.index("Depósito / Bodega")
        features[27] = 1
    except:
        features[27] = 0


    return features


In [ ]:
# #Selenium+BS
# pag = 2
# browser = uc.Chrome()
# url = f'https://www.fincaraiz.com.co/apartamentos-casas/venta/bogota/bogota-dc?pagina={pag}'
# print(url)
# browser.get(url)
# browser.implicitly_wait(10)
# html = browser.page_source
# soup = bs(html,'lxml')

In [ ]:
# id_inmuebles = []

# #Get href
# links = gethref(soup)

# #Remove "Proyectos de vivienda"
# id_inmuebles = remove_proyectos(links)

In [ ]:
# #Scraping
# cols = ['habitaciones','baños','parqueaderos','area_construida','area_privada','estrato','estado','antiguedad',
#         'administracion','precio_m2', 'Ascensor', 'Circuito cerrado de TV',
#        'Parqueadero Visitantes', 'Portería / Recepción', 'Zonas Verdes', 'Salón Comunal', 'Balcón',
#        'Barra estilo americano', 'Calentador', 'Chimenea', 'Citófono', 'Cocina Integral', 'Terraza',
#        'Vigilancia', 'Parques cercanos', 'Estudio', 'Patio', 'Depósito / Bodega', 'nombre','ubicacion','precio']

# data = pd.DataFrame(columns=cols)

# p = []
# #Scraping a los inmuebles filtrados
# for i in range(len(id_inmuebles)):
# # for i in range(0,3):
#     print('Scrapping', i, '/', len(id_inmuebles), '...')
#     p.append(scrapper(id_inmuebles[i]))
#     print(p[i])

#     #append list to DataFrame
#     data.loc[len(data)] = p[i]

In [ ]:
# data

#### Initialize and execute Selenium + BeautifulSoap

In [7]:
from lxml import etree
import sys

In [8]:
browser = uc.Chrome()

links = []
aux = []
id_inmuebles = []

cols = ['habitaciones','baños','parqueaderos','area_construida','area_privada','estrato','estado','antiguedad',
        'administracion','precio_m2', 'Ascensor', 'Circuito cerrado de TV',
       'Parqueadero Visitantes', 'Portería / Recepción', 'Zonas Verdes', 'Salón Comunal', 'Balcón',
       'Barra estilo americano', 'Calentador', 'Chimenea', 'Citófono', 'Cocina Integral', 'Terraza',
       'Vigilancia', 'Parques cercanos', 'Estudio', 'Patio', 'Depósito / Bodega', 'nombre','ubicacion','precio']
data = pd.DataFrame(columns=cols)

In [10]:
def scrape_pages(browser, min, max):
  #Range of pages from the total search to scrape in.
  #It is recomended to cover a range of one hundred pages in each iteration of this section.
  for i in range(min,max):

      pag = i

      url = f'https://www.fincaraiz.com.co/apartamentos-casas/venta/bogota/bogota-dc?pagina={pag}'
      print(url)
      browser.get(url)
      browser.implicitly_wait(10)
      html = browser.page_source
      soup = bs(html,'lxml')

  #     #Obtaining actual page by using full xpath
  #     dom = etree.HTML (str(soup))
  #     if i < 5:
  #         print('Con if, i es igual a: ',i)
  #         pagina_actual=dom.xpath("".join(['/html/body/div[1]/div[1]/div[3]/div/div[1]/div[4]/div[1]/nav/ul/li[', str(i+1), ']/button']))[0].text
  #         print(int(pagina_actual))
  #     else:
  #         print('Con else, i es igual a: ',i)
  #         pagina_actual=dom.xpath('/html/body/div[1]/div[1]/div[3]/div/div[1]/div[4]/div[1]/nav/ul/li[5]/button')[0].text
  #         print(int(pagina_actual))

      #Get href
      links = gethref(soup)

      #Remove "Proyectos de vivienda"
      id_inmuebles = []
      id_inmuebles = remove_proyectos(links)

      #Scrapping
      p = []
      #Scrapping a los inmuebles filtrados
      for i in range(len(id_inmuebles)):
      # for i in range(0,29):
          print('Scrapping', i, '/', len(id_inmuebles), '...')
          p.append(scrapper(id_inmuebles[i]))
          print(p[i])

          #append list to DataFrame
          data.loc[len(data)] = p[i]
  return data

In [12]:
data = scrape_pages(browser, 0, 10)

https://www.fincaraiz.com.co/apartamentos-casas/venta/bogota/bogota-dc?pagina=0
Href obtained:  44
Articles '/proyecto-de-vivienda' identified:  19
Scrapping 0 / 25 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/nuevo-techo/bogota/8023393/
['3', '2', '1', '73 m²', '73 m²', '2', 'No definida', '9 a 15 años', '$\xa0172.300 COP', '$\xa04.109.589,04*m²', 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Nuevo Techo', 300000000]
Scrapping 1 / 25 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/maria-cristina/bogota/10002420/
['2', '3', '1', '85 m²', '93 m²', '5', 'No definida', '16 a 30 años', '$\xa0410.000 COP', '$\xa04.470.588,24*m²', 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 'Apartamento', 'Maria cristina', 380000000]
Scrapping 2 / 25 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/mazuren/bogota/10117928/
['3', '2', '1', '80 m²', '0 m²', '4', 'No definida', '9 a 15 años', 'No definida', '$\xa04.987.500*m²', 1, 

['3', '2', '1', '78 m²', '69 m²', '4', 'Excelente', '1 a 8 años', '$\xa0320.000 COP', '$\xa07.051.282,05*m²', 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Caobos salazar', 550000000]
Scrapping 1 / 25 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/corredor-ferreo-del-sur/bogota/10082616/
['2', '1', '0', '43 m²', '0 m²', '3', 'No definida', 'No definida', '$\xa0125.000 COP', '$\xa04.883.720,93*m²', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Corredor ferreo del sur', 210000000]
Scrapping 2 / 25 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/country-club/bogota/8016936/
['1', '2', '1', '64 m²', '0 m²', '5', 'No definida', '1 a 8 años', '$\xa0499.000 COP', '$\xa06.015.625*m²', 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Country club', 385000000]
Scrapping 3 / 25 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/jardin-del-norte/bogota/10104036/
['3', '2', '0', '60,2 m²'

['3', '3', '0', '106 m²', '106 m²', '3', 'Bueno', '16 a 30 años', '$\xa01 COP', '$\xa02.452.830,19*m²', 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 'Casa', 'Gran granada', 260000000]
Scrapping 2 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/la-salle/bogota/7883475/
['2', '2', '0', '67 m²', '0 m²', '4', 'Excelente', '16 a 30 años', '$\xa0391.000 COP', '$\xa06.014.925,37*m²', 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'La salle', 403000000]
Scrapping 3 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/puerta-de-teja/bogota/10128402/
['3', '2', '1', '64 m²', '64 m²', '4', 'No definida', '16 a 30 años', '$\xa0196.000 COP', '$\xa04.843.750*m²', 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Puerta de teja', 310000000]
Scrapping 4 / 30 ...
https://www.fincaraiz.com.co/inmueble/casa-en-venta/la-calleja/bogota/7985011/
['4', '4', '3', '245 m²', '240 m²', '5', 'Bueno', '16 a 30 años', '$\xa0450.00

https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/centro-comercial-palatino/bogota/10124448/
['3', '4', '3', '186 m²', '168 m²', '5', 'No definida', '16 a 30 años', '$\xa01.100.000 COP', '$\xa06.881.720,43*m²', 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Centro Comercial Palatino', 1280000000]
Scrapping 29 / 30 ...
https://www.fincaraiz.com.co/inmueble/casa-en-venta/quinta-camacho/bogota/7161474/
['8', '6', '3', '350 m²', '253 m²', '4', 'Bueno', '16 a 30 años', 'No definida', '$\xa03.854.285,71*m²', 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 'Casa', 'Quinta camacho', 1349000000]
https://www.fincaraiz.com.co/apartamentos-casas/venta/bogota/bogota-dc?pagina=3
Href obtained:  44
Articles '/proyecto-de-vivienda' identified:  14
Scrapping 0 / 30 ...
https://www.fincaraiz.com.co/inmueble/casa-en-venta/modelia/bogota/7855144/
['4', '3', '4', '145 m²', '112 m²', '4', 'No definida', '9 a 15 años', 'No definida', '$\xa05.551.724,14*m²', 0, 1, 0, 0, 1

https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/colina-campestre/bogota/7256564/
['3', '3', '2', '152 m²', '152 m²', '5', 'Excelente', '1 a 8 años', 'No definida', '$\xa06.250.000*m²', 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Colina Campestre', 950000000]
Scrapping 25 / 30 ...
https://www.fincaraiz.com.co/inmueble/casa-en-venta/chico-navarra/bogota/7111473/
['4', '5', '2', '175 m²', '172 m²', '6', 'Bueno', 'más de 30 años', '$\xa0998.000 COP', '$\xa05.428.571,43*m²', 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 'Casa', 'Chicó Navarra', 950000000]
Scrapping 26 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/julio-florez/bogota/7109670/
['3', '2', '2', '112 m²', '110 m²', '6', 'Bueno', '16 a 30 años', '$\xa0845.000 COP', '$\xa05.625.000*m²', 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 'Apartamento', 'Julio florez', 630000000]
Scrapping 27 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/la-c

...

https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/rincon-del-chico/bogota/7045930/
['5', '5', '4', '300 m²', '300 m²', '6', 'Excelente', '16 a 30 años', '$\xa01.550.000 COP', '$\xa04.833.333,33*m²', 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Rincon del chico', 1450000000]
Scrapping 13 / 30 ...
https://www.fincaraiz.com.co/inmueble/casa-en-venta/pradera-norte/bogota/7447680/
['4', '3', '1', '120 m²', '0 m²', '4', 'Excelente', '16 a 30 años', '$\xa0300.000 COP', '$\xa03.491.666,67*m²', 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 'Casa', 'Pradera Norte', 419000000]
Scrapping 14 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/bella-suiza/bogota/8079418/
['3', '2', '1', '57 m²', '57 m²', '5', 'Bueno', 'más de 30 años', '$\xa0336.000 COP', '$\xa04.719.298,25*m²', 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 'Apartamento', 'Bella Suiza', 269000000]
Scrapping 15 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento

https://www.fincaraiz.com.co/inmueble/casa-en-venta/fontibon/bogota/7758814/
['3', '2', '0', '59 m²', '59 m²', '2', 'No definida', '16 a 30 años', '$\xa048.000 COP', '$\xa02.593.220,34*m²', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 'Casa', 'Fontibón', 153000000]
Scrapping 9 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/canodromo/bogota/10126142/
['4', '4', '2', '186,91 m²', '186,91 m²', '5', 'No definida', 'más de 30 años', '$\xa0945.000 COP', '$\xa06.687.710,66*m²', 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Canodromo', 1250000000]
Scrapping 10 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/victoria-norte/bogota/7984713/
['1', '1', '0', '45 m²', '45 m²', '4', 'No definida', '16 a 30 años', '$\xa0303.000 COP', '$\xa04.155.555,56*m²', 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Victoria Norte', 187000000]
Scrapping 11 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-v

https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/el-encanto/bogota/8068317/
['3', '2', '1', '75 m²', '75 m²', '4', 'No definida', '1 a 8 años', '$\xa0452.000 COP', '$\xa04.813.333,33*m²', 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'El Encanto', 361000000]
Scrapping 5 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/kennedy/bogota/8019393/
['2', '1', '0', '51 m²', '51 m²', '3', 'No definida', '1 a 8 años', '$\xa095.000 COP', '$\xa02.823.529,41*m²', 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Kennedy', 144000000]
Scrapping 6 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/bosque-medina/bogota/7747501/
['4', '5', '3', '262 m²', '0 m²', '6', 'No definida', '16 a 30 años', '$\xa01.250.000 COP', '$\xa07.099.236,64*m²', 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Bosque Medina', 1860000000]
Scrapping 7 / 30 ...
https://www.fincaraiz.com.co/inmueble/casa-en-venta/marly

Href obtained:  44
Articles '/proyecto-de-vivienda' identified:  14
Scrapping 0 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/chico/bogota/10079893/
['2', '2', '2', '142 m²', '0 m²', '6', 'No definida', '9 a 15 años', '$\xa01.349.000 COP', '$\xa09.154.929,58*m²', 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 'Apartamento', 'Chico', 1300000000]
Scrapping 1 / 30 ...
https://www.fincaraiz.com.co/inmueble/casa-en-venta/ciudad-bachue-i-etapa/bogota/10093102/
['8', '5', '0', '70 m²', '70 m²', '2', 'No definida', 'más de 30 años', 'No definida', '$\xa04.285.714,29*m²', 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 'Casa', 'Ciudad bachue i etapa', 300000000]
Scrapping 2 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/los-rosales/bogota/7519117/
['3', '5', '2', '158 m²', '158 m²', '6', 'No definida', '9 a 15 años', '$\xa01.250.000 COP', '$\xa08.227.848,1*m²', 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 'Apartamento', 'Los Rosal

https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/santa-paula/bogota/8086978/
['3', '5', '3', '257 m²', '0 m²', '6', 'No definida', '16 a 30 años', '$\xa01.549.900 COP', '$\xa06.809.338,52*m²', 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 'Apartamento', 'Santa Paula', 1750000000]
Scrapping 27 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/cedritos/bogota/7972175/
['3', '4', '2', '129 m²', '0 m²', '5', 'Excelente', '16 a 30 años', '$\xa0698.000 COP', '$\xa05.271.317,83*m²', 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 'Apartamento', 'Cedritos', 680000000]
Scrapping 28 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/villa-alsacia/bogota/10064548/
['3', '2', '1', '96 m²', '0 m²', '4', 'No definida', '1 a 8 años', '$\xa0290.000 COP', '$\xa05.833.333,33*m²', 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 'Apartamento', 'Villa alsacia', 560000000]
Scrapping 29 / 30 ...
https://www.fincaraiz.com.co/inmueble/apartamento

In [ ]:
data.to_csv('housing_fincaraiz_example.csv', encoding='utf-8', index=False)

## Referencias

https://github.com/kiteco/kite-python-blog-post-code/blob/master/Web%20Scraping%20Tutorial/script.py

https://medium.com/geekculture/scrappy-guide-to-web-scraping-with-python-475385364381

https://stackoverflow.com/questions/47730671/python-3-using-requests-does-not-get-the-full-content-of-a-web-page